# NLU Group 1: Final Project
## Example: Fine-tuning a classifier on the Social Bias Inference Corpus

Based on: https://github.com/huggingface/notebooks/blob/master/examples/text_classification.ipynb

This notebook by: Cameron Clarke (ccc779@nyu.edu)

Created: 3/16/2021

# 0. Setup

In [ ]:
!pip install transformers
!pip install datasets
!pip install bert_score
!pip install optuna

     |████████████████████████████████| 2.0MB 5.8MB/s 
     |████████████████████████████████| 890kB 18.4MB/s 
     |████████████████████████████████| 3.2MB 26.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=098d5a143ba85724ce543ad5db0f142f9ff1780b0d570cf7538cf19e8e520c38
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 194kB 6.0MB/s 
     |████████████████████████████████| 112kB 14.0MB/s 
     |████████████████████████████████| 245kB 11.8MB/s 
     |████████████████████████████████| 61kB 3.3MB/s 
     |████████████████████████████████| 296kB 4.4MB/s 
     |████████████████████████████████| 163kB 43.1MB/s 
     |████████████████████████████████| 81kB 11.3MB/s 
     |████████████████████████████████| 81kB 11.0MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
     |████████████████████████████████|

# 1. Preprocessing

In [ ]:
from transformers import AutoTokenizer
import datasets
from datasets import load_dataset, load_metric
import numpy as np
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
# [TODO] update this for whichever model type is desired

model_checkpoint = "bert-base-cased"
batch_size = 8

In [ ]:
# [TODO] update this for whichever dataset is desired

dataset = load_dataset("social_bias_frames")

Using custom data configuration default


Dataset social_bias_frames downloaded and prepared to /root/.cache/huggingface/datasets/social_bias_frames/default/0.0.0/7ccf5e07dabdba6791693ea27289996d4771f586aa88f1ff05c52645f2cfd41d. Subsequent calls will reuse this data.


In [ ]:
metric = load_metric('bertscore')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [ ]:
sentence1_key = 'post'

In [ ]:
def preprocess_function(examples, sentence1_key, sentence2_key=None):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True, padding=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True, padding=True)

In [ ]:
label_colname = 'offensiveYN'

relabel_dict = {
    '0.0': 0, # not offensive
    '0.5': 1, # maybe offiensive
    '1.0': 2, # offensive
    '': None # missing value
}

relabel_func = lambda column: [relabel_dict[elt] for elt in column]

In [ ]:
# Following a design pattern found here: https://huggingface.co/transformers/custom_datasets.html#using-the-nlp-datasets-metrics-library


encoded_dataset = dataset.map(lambda x: preprocess_function(x, sentence1_key),  batched=True)
encoded_dataset = encoded_dataset.map(lambda x: {'labels': relabel_func(x[label_colname])},  batched=True)

new_features = encoded_dataset['train'].features.copy()
new_features["labels"] = datasets.ClassLabel(names=['no', 'maybe', 'yes'])

encoded_dataset['train'] = encoded_dataset['train'].cast(new_features)
encoded_dataset['validation'] = encoded_dataset['validation'].cast(new_features)
encoded_dataset['test'] = encoded_dataset['test'].cast(new_features)



In [ ]:
encoded_dataset = encoded_dataset.filter(lambda row: not (row['labels'] is None))

In [ ]:
encoded_dataset['train']['offensiveYN'], encoded_dataset['train']['labels']

(['1.0',
  '0.5',
  '0.5',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '0.0',
  '0.0',
  '0.0',
  '1.0',
  '1.0',
  '0.5',
  '0.5',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '0.0',
  '1.0',
  '1.0',
  '0.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '0.5',
  '0.5',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '0.0',
  '1.0',
  '0.0',
  '1.0',
  '1.0',
  '0.0',
  '0.5',
  '0.0',
  '1.0',
  '1.0',
  '0.0',
  '0.5',
  '0.5',
  '1.0',
  '0.0',
  '1.0',
  '0.5',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '0.5',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '0.0',
  '0.0',
  '1.0',
  '0.0',
  '1.0',
  '0.5',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '0.5',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.5',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
 

# 2. Fine-tuning

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

In [ ]:
# num_labels = 3 # The original 'offensiveYN' outcome is a categorical variable with three levels
# model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
# model.eval()
# model.to('cuda')

In [ ]:
# [TODO] update this with whatever metric is desired

metric_name = "accuracy"

args = TrainingArguments(
    "test-SBIC-bert",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

In [ ]:
# Reference: https://github.com/armandalewis/ling-ga-1012/blob/main/eval_metrics.py

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # precision, recall, f1, _ = precision_recall_fscore_support(labels, preds)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        # 'f1': f1,
        # 'precision': precision,
        # 'recall': recall
    }

# def compute_metrics(eval_pred, metric):
#     predictions, labels = eval_pred

#     predictions = np.argmax(predictions, axis=1)
#     return metric.compute(predictions=predictions, references=labels)

In [ ]:
# validation_key = "validation"
# trainer = Trainer(
#     model,
#     args,
#     train_dataset=encoded_dataset['train'],
#     eval_dataset=encoded_dataset['validation'],
#     tokenizer=tokenizer,
#     compute_metrics=lambda x: compute_metrics(x, metric)
# )

# Hyperparameter Tuning

In [ ]:
def model_init():
    num_labels = 3 # The original 'offensiveYN' outcome is a categorical variable with three levels
    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
    model.eval()
    model.to('cuda')
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

In [ ]:
trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['validation'],
    tokenizer=tokenizer,
    # compute_metrics=lambda x: compute_metrics(x, metric)
    compute_metrics=compute_metrics
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
def my_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 1e-2, log=True),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 1, 1),
        "seed": trial.suggest_int("seed", 1, 40),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [batch_size]),
    }

In [ ]:
best_run = trainer.hyperparameter_search(n_trials=5, direction="maximize", hp_space=my_hp_space)

[I 2021-03-31 12:40:43,616] A new study created in memory with name: no-name-9a9d8dc1-63d3-439b-833a-c64f96296739
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassi

Epoch,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
1,0.955200,0.900342,0.573074,94.162400,175.197000


[I 2021-03-31 13:38:21,701] Trial 0 finished with value: 0.5730738922228283 and parameters: {'learning_rate': 0.0017083372941272326, 'num_train_epochs': 1, 'seed': 14, 'per_device_train_batch_size': 8}. Best is trial 0 with value: 0.5730738922228283.
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the check

Epoch,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
1,0.985900,0.904427,0.573074,94.679700,174.240000


[I 2021-03-31 14:36:17,091] Trial 1 finished with value: 0.5730738922228283 and parameters: {'learning_rate': 0.009984450997581298, 'num_train_epochs': 1, 'seed': 33, 'per_device_train_batch_size': 8}. Best is trial 0 with value: 0.5730738922228283.
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkp

...

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
1,0.979000,0.894108,0.573074,94.419500,174.720000


[I 2021-03-31 16:32:15,496] Trial 3 finished with value: 0.5730738922228283 and parameters: {'learning_rate': 0.005465576677694313, 'num_train_epochs': 1, 'seed': 26, 'per_device_train_batch_size': 8}. Best is trial 0 with value: 0.5730738922228283.
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkp

Epoch,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
1,0.940700,0.901132,0.573074,94.304600,174.933000


[I 2021-03-31 17:30:17,072] Trial 4 finished with value: 0.5730738922228283 and parameters: {'learning_rate': 0.0019008891007900582, 'num_train_epochs': 1, 'seed': 15, 'per_device_train_batch_size': 8}. Best is trial 0 with value: 0.5730738922228283.


In [ ]:
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Epoch,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
1,0.955200,0.900342,0.573074,94.325800,174.894000


TrainOutput(global_step=13861, training_loss=1.0214273050813376, metrics={'train_runtime': 3475.5147, 'train_samples_per_second': 3.988, 'total_flos': 1.8257329918607868e+16, 'epoch': 1.0, 'train_mem_cpu_alloc_delta': 1960809, 'train_mem_gpu_alloc_delta': 13911040, 'train_mem_cpu_peaked_delta': 89718129, 'train_mem_gpu_peaked_delta': 6059090432})

In [ ]:
trainer.evaluate()

{'epoch': 1.0,
 'eval_accuracy': 0.5730738922228283,
 'eval_loss': 0.9003417491912842,
 'eval_mem_cpu_alloc_delta': 405831,
 'eval_mem_cpu_peaked_delta': 1172198,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 203021312,
 'eval_runtime': 94.2749,
 'eval_samples_per_second': 174.988}

In [ ]:
trainer.state

TrainerState(epoch=1.0, global_step=13861, max_steps=13861, num_train_epochs=1, total_flos=1.8257329918607868e+16, log_history=[{'loss': 1.2008, 'learning_rate': 0.001646713410780893, 'epoch': 0.04, 'step': 500}, {'loss': 1.0669, 'learning_rate': 0.0015850895274345529, 'epoch': 0.07, 'step': 1000}, {'loss': 1.1307, 'learning_rate': 0.0015234656440882132, 'epoch': 0.11, 'step': 1500}, {'loss': 1.0691, 'learning_rate': 0.0014618417607418733, 'epoch': 0.14, 'step': 2000}, {'loss': 1.0785, 'learning_rate': 0.0014002178773955337, 'epoch': 0.18, 'step': 2500}, {'loss': 1.0436, 'learning_rate': 0.0013385939940491936, 'epoch': 0.22, 'step': 3000}, {'loss': 1.0641, 'learning_rate': 0.0012769701107028539, 'epoch': 0.25, 'step': 3500}, {'loss': 1.0455, 'learning_rate': 0.001215346227356514, 'epoch': 0.29, 'step': 4000}, {'loss': 1.0324, 'learning_rate': 0.0011537223440101743, 'epoch': 0.32, 'step': 4500}, {'loss': 1.0252, 'learning_rate': 0.0010920984606638342, 'epoch': 0.36, 'step': 5000}, {'los